In [1]:
class DecoderState_ensemble(object):
    """
    State of Decoder.
    """

    def __init__(self, states):
        """
        hidden: Tensor(num_layers, batch_size, hidden_size)
        """
        self.states=states
        self.mask=states[0].mask
        

    def __getattr__(self, name):
        return self.__dict__.get(name)

    def set_mask(self,):
        for state in self.states:
            state.mask=self.mask
    
    def get_batch_size(self):
        """
        get_batch_size
        """
        return self.states[0].get_batch_size()

    def size(self):
        """
        size
        """
        return self.states[0].size()

    def slice_select(self, stop):
        """
        slice_select
        """
        states=[]
        for state in self.states:
            states.append(state.slice_select(stop))
        return DecoderState_ensemble(states)

    def index_select(self, indices):
        """
        index_select
        """
        states=[]
        for state in self.states:
            states.append(state.index_select(indices))
        return DecoderState_ensemble(states)

    def mask_select(self, mask):
        """
        mask_select
        """
        states=[]
        for state in self.states:
            states.append(state.mask_select(mask))
        return DecoderState_ensemble(states)

    def inflate(self, times):
        """
        inflate
        """
        states=[]
        for state in self.states:
            states.append(state.inflate(times))
            
        return DecoderState_ensemble(states)





In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
from source.utils.misc import Pack

In [5]:
class ensemble(nn.Module):
    def __init__(self, models):
        super(ensemble, self).__init__()
        self.num=len(models)
        self.models=nn.ModuleList(models)
    def encode(self,inputs, enc_hidden):
        enc_outputs=Pack()
        states=[]
        for i in range(self.num):
            _, state = self.models[i].encode(inputs, enc_hidden)
            states.append(state)
#         print(len(states))
        return enc_outputs, DecoderState_ensemble(states)
    
    def decode(self,input_var, dec_state):
#         print('-----')
        dec_state.set_mask()
        p=Pack()
        output_temp=[]
        attn_temp=[]
        dec_state_temp=[]
#         print(len(dec_state.states))
        for i in range(self.num):
#             state_=dec_state.states[i]
#             model_=self.models[i]
#             print(type(state_))
#             print(type(model_))
#             output, dec_state_0, attn = model_.decode(input_var, state_)
            output, dec_state_0, attn = self.models[i].decode(input_var, dec_state.states[i])
            output_temp.append(output)
            attn_temp.append(attn.attn)
            dec_state_temp.append(dec_state_0)
        
        output_temp=torch.stack(output_temp).mean(0)
        attn_temp=torch.stack(attn_temp).mean(0)
        p.add(attn=attn_temp)
        dec_state_temp=DecoderState_ensemble(dec_state_temp)
        return output_temp, dec_state_temp, p
            
            
            

In [6]:
file='./models/layer_2_64_embed_elmo/params.json'

In [7]:
import json

In [8]:
p_file=open(file,'r',encoding='utf-8')
pp=json.loads(p_file.read())
config=Pack(pp)

In [9]:
config.save_dir+'/best.model'

'./models/layer_2_64_embed_elmo/best.model'

In [10]:
from source.inputters.corpus import KnowledgeCorpus, Entity_Corpus, Entity_Corpus_pos

In [11]:
config.gpu=0
device = config.gpu
torch.cuda.set_device(device)

In [12]:
corpus = Entity_Corpus_pos(data_dir=config.data_dir, data_prefix=config.data_prefix, entity_file=config.entity_file,
                             min_freq=config.min_freq, max_vocab_size=config.max_vocab_size)
corpus.load()

加载实体成功，size= 321422
Loading prepared vocab from ./data/data550_pos/model_50000.vocab.pt ...
Finish loading vocab , size: 50005
Finish loading pos vocab , size: 31
Loading prepared data from ./data/data550_pos/model_50000.data.pt ...
Number of examples: TRAIN-37226 VALID-2000


In [13]:
# valid_iter = corpus.create_batches(
#         config.batch_size, "valid", shuffle=False, device=device)

In [13]:
valid_iter = corpus.create_batches(
    5, "valid", shuffle=False, device=device)

In [15]:
from tqdm import tqdm
from source.models.Entity_seq2seq import Entity_Seq2Seq
from source.models.Entity_seq2seq_pos import Entity_Seq2Seq_pos
from source.models.Entity_seq2seq_pos_gru import Entity_Seq2Seq_pos_gru
from source.models.Entity_seq2seq_pos_elmo import Entity_Seq2Seq_elmo
from source.utils.generator import TopKGenerator
from source.utils.engine import evaluate, evaluate_generation

ModuleNotFoundError: No module named 'source.models.Entity_seq2seq_pos_elmo'

In [15]:

file1='./models/layer_2_64_embed_10/params.json'
file2='./models/layer_2_64_embed_11/params.json'
file9='./models/layer_2_64_embed_13/params.json'
file10='./models/layer_2_64_embed_14/params.json'
file11='./models/layer_2_64_embed_15/params.json'
file12='./models/layer_2_64_embed_16/params.json'
files=[file1,file2,file9,file10,file11,file12]

def get_model(files):
    models=[]
    for f in tqdm(files):
        p_file=open(f,'r',encoding='utf-8')
        pp=json.loads(p_file.read())
        config=Pack(pp)
        model = model = Entity_Seq2Seq_pos(src_vocab_size=corpus.SRC.vocab_size,
                               pos_vocab_size=corpus.POS.vocab_size,
                               embed_size=config.embed_size, hidden_size=config.hidden_size,
                               padding_idx=corpus.padding_idx,
                               num_layers=config.num_layers, bidirectional=config.bidirectional,
                               attn_mode=config.attn, with_bridge=config.with_bridge,
                               dropout=config.dropout,
                               use_gpu=config.use_gpu,
                               pretrain_epoch=config.pretrain_epoch)
        config.ckpt=config.save_dir+'/best.model'
        model.load(config.ckpt)
        models.append(model)
    return ensemble(models)

# models=get_model(files)

In [19]:
models=get_model(files)

 25%|██▌       | 1/4 [00:00<00:02,  1.48it/s]

Loaded model state from './models/layer_2_64_embed_18/best.model'


 50%|█████     | 2/4 [00:01<00:01,  1.36it/s]

Loaded model state from './models/layer_2_64_embed_19/best.model'


 75%|███████▌  | 3/4 [00:02<00:00,  1.24it/s]

Loaded model state from './models/layer_2_64_embed_20/best.model'


100%|██████████| 4/4 [00:03<00:00,  1.19it/s]

Loaded model state from './models/layer_2_64_embed_21/best.model'


In [20]:
config.max_dec_len=5
config.beam_size=5
generator = TopKGenerator(model=models,
                              src_field=corpus.SRC,
                              max_length=config.max_dec_len, ignore_unk=config.ignore_unk,
			      length_average=config.length_average, use_gpu=config.use_gpu, beam_size=config.beam_size)

In [21]:
evaluate_generation(generator, valid_iter, save_file=config.gen_file, verbos=True)

Saved generation results to './test.result'
Avg_Len-2.293   F1-0.627   precision-0.643   recal-0.686   emo_precision-0.647
Target:   AVG_LEN-2.093   


In [22]:
corpus.reload(data_type='test')
test_iter = corpus.create_batches(
            20, "test", shuffle=False, device=device)

Number of examples: TRAIN-37226 VALID-2000 TEST-40000


In [23]:
evaluate_generation(generator, test_iter, save_file=config.gen_file, verbos=True, for_test=True)

Avg_Len-2.299   F1-0.000   precision-0.000   recal-0.000   emo_precision-0.000
Target:   AVG_LEN-1.000   


In [21]:
Saved generation results to './test.result'
Avg_Len-2.179   F1-0.631   precision-0.661   recal-0.674   emo_precision-0.635
Target:   AVG_LEN-2.093   

SystemExit: 

/home/dutir923/irguest/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [27]:
Saved generation results to './test.result'
Avg_Len-2.182   F1-0.633   precision-0.661   recal-0.676   emo_precision-0.637
Target:   AVG_LEN-2.093   

4